# Sous problème - Par modélisation PLNE

Dans ce notebook, nous définissons la première méthode de résolution du sous problème de la génération de colonnes.

## Création du modèle

In [ ]:
function sub_pb_PLNE(A,W,p,n,K,preds,succ)
    """
    Crée et résout le sous-problème de la génération de colonnes.

    Entrées :
        - A     : Array   -  Matrice des arcs du graphe
        - W     : Array   -  Matrice des poids de chaque arc
        - p     : Array   -  Solution optimale du dual de la relaxation
        - n     : Integer -  Nombre de sommets dans le graphe
        - K     : Integer -  Taille maximale des cycles
        - preds : Array   - Liste des prédecesseurs de chaque sommet
        - succ  : Array   - Liste des successeurs de chaque sommet
    
    Sorties : 
        - model : JuMP Model - Modèle résolu
    """
    # Vecteur des sommets
    V = Array(1:n)
        
    # JuMP model
    model = Model(Cbc.Optimizer); 
    set_optimizer_attribute(model, "logLevel", 0)
    
    # Variables
    ## Pour tout arc "a" de A, y[a] = 1 si l'arc fait partie du cycle e coût maximal
    @variable(model, y[A], Bin);

    # Fonction objectif: maximiser le coût du cycle choisi
    @objective(model, Max, sum(y[(i,j)]*(W[i,j]-p[j]) for (i,j) in A));
    
    # Contraintes
    ## Un couple peut donner, au plus, à un seul autre couple
    @constraint(model,ct_flot[i in V],sum(y[(i,j)] for j in succ[i])<=1)
    ## Si un couple donne, alors il doit aussi recevoir
    @constraint(model,ct_flot2[i in V],sum(y[(i,j)] for j in succ[i])==sum(y[(j,i)] for j in preds[i]))
    ## Le cycle choisi doit être de taille <= K
    @constraint(model,ct_length,sum(y[(i,j)] for (i,j) in A)<=K)
    
    timer = @timed optimize!(model);  

    return model;
end

## Fonction construisant les cycles solution

In [ ]:
function compute_solution_PLNE(model,W,p,eps)
    """
    Crée les cycles solutions rendu par la modélisation en PLNE.
    
    Entrées : 
        - model      : JuMP Model - Modèle résolu du problème
        - W          : Array      - Matrice des poids des arcs
        - p          : Array      - Vecteur des variables duales optimales
        - eps        : Float      - Marge d'erreur prise en compte dans la comparaison à 0
    
    Sorties : 
        - cycle : Array - Cycle de coût positif
    """
    # Récupération des valeurs optimales des variables 
    X = value.(model[:y])

    # Récupération des arcs solution
    arcs = []
    for (i,j) in axes(X)[1]
        if X[(i,j)] == 1
            push!(arcs,(i,j))
        end
    end
    
    if length(arcs) > 0
        # Initialisation du cycle solution
        cycle = [] 
        next = arcs[1]
        # Initialisation de son coût
        cout_cycle = 0

        # Tant qu'il reste des arcs à traiter
        while true
            # On ajoute l'arc au cycle courant
            push!(cycle,next)
            cout_cycle += W[next[1],next[2]] - p[next[2]]

            # Et on le supprime de la liste des arcs à traiter
            arcs = filter(t -> t != next,arcs)

            # On met à jour l'arc suivant
            next = filter(t -> next[2] in t, arcs)
            if length(next) > 0
                # Cas où le cycle courant n'est pas terminé
                next = next[1]
            else
                # Cas où le cycle courant est terminé
                # Si le cycle est de coût positif, alors on le renvoie
                if cout_cycle > eps
                    return cycle
                else
                    # Si le cycle est de coût négatif, cela signifie que les autres cycles le sont aussi
                    # --> On ne renvoie rien
                    return nothing
                end
                
            end
        end       
    else
        # Si aucun cycle n'est trouvé, on ne renvoie rien
       return nothing     
    end
end

In [1]:
println("Résolution par PLNE importée avec succès.")

Résolution par PLNE importée avec succès.
